In [11]:
import numpy as np
import chaospy as cp

distribution = cp.Uniform(0, 15)
samples = distribution.sample(10, rule="sobol")
evaluations = samples*np.sin(samples)

expansion = cp.generate_expansion(9, distribution, normed=True)

In [12]:
from sklearn.linear_model import LarsCV

lars = LarsCV(fit_intercept=False, max_iter=5)
pce, coeffs = cp.fit_regression(
    expansion, samples, evaluations, model=lars, retall=True)
expansion_ = expansion[coeffs != 0]

lars = LarsCV(fit_intercept=False, max_iter=5)
lars.fit(expansion(samples).T, evaluations)
expansion_ = expansion[lars.coef_ != 0]


In [13]:
expansion_, lars.coef_

(polynomial([0.01567852628779017*q0**3-0.35276684147527876*q0**2+2.1166010488516727*q0-2.6457513110645907,
             0.0011006280785771994*q0**5-0.04127355294664498*q0**4+0.5503140392885997*q0**3-3.095516470998373*q0**2+6.6332495807108005*q0-3.3166247903554007,
             0.0002924799454401076*q0**6-0.013161597544804842*q0**5+0.2243454126955371*q0**4-1.7947633015642965*q0**3+6.7303623808661115*q0**2-10.09554357129917*q0+3.6055512754639905,
             7.779543173550678e-05*q0**7-0.004084260166114106*q0**6+0.08482694191160066*q0**5-0.8836139782458402*q0**4+4.819712608613674*q0**3-13.013224043256919*q0**2+14.459137825841022*q0-3.8729833462074184,
             5.512776981003564e-06*q0**9-0.0003721124462177405*q0**8+0.010506704363795026*q0**7-0.16088391057061135*q0**6+1.4479551951355019*q0**5-7.756902831083046*q0**4+23.86739332640937*q0**3-38.35831070315793*q0**2+26.153393661244042*q0-4.358898943540676]),
 array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  5.55014633e-01,
   

In [14]:
expansion_(np.linspace(0,15,100).reshape(-1,1))

array([[[-2.64575131],
        [-2.33309806],
        [-2.03631443],
        [-1.7550732 ],
        [-1.48904717],
        [-1.23790913],
        [-1.00133187],
        [-0.77898819],
        [-0.57055087],
        [-0.37569271],
        [-0.1940865 ],
        [-0.02540502],
        [ 0.13067893],
        [ 0.27449255],
        [ 0.40636306],
        [ 0.52661767],
        [ 0.63558358],
        [ 0.733588  ],
        [ 0.82095815],
        [ 0.89802122],
        [ 0.96510443],
        [ 1.02253499],
        [ 1.07064011],
        [ 1.10974698],
        [ 1.14018284],
        [ 1.16227487],
        [ 1.17635029],
        [ 1.18273631],
        [ 1.18176014],
        [ 1.17374898],
        [ 1.15903005],
        [ 1.13793055],
        [ 1.11077769],
        [ 1.07789868],
        [ 1.03962073],
        [ 0.99627105],
        [ 0.94817684],
        [ 0.89566532],
        [ 0.83906369],
        [ 0.77869916],
        [ 0.71489894],
        [ 0.64799024],
        [ 0.57830026],
        [ 0